# Gradient based optimisation

## Prerequisites

- Calculus
- Linear Regression Introduction

## Learning objectives

- Understand the concepts behind gradient based optimisation and learning
- Implement the stochastic gradient descent (SGD) algorithm from scratch in Python
- Implement linear regression from scratch in Python

## Intro

Previously, we saw how we could optimise parameters of linear regression using it's analytical solutions. This technique had some fundamental flaws though...
- It is available only for special cases of machine learning methods, hence __isn't general method__
- Computationally infeasible for large datasets and a lot of features

This notebook will walk through how we can use **gradient based optimisation** as another technique to find model parameterisations that perform well.

Gradient based optimisation is not exclusively used in machine learning, it is a technique used for optimising all kinds of functions in all kinds of domains.

> We are going to use gradient based optimisation to minimise the criterion of our ML algorithms

To get your mind running, gradient based optimisation looks a bit like the following diagram. In our case, $f(w)$ would represent the criterion that we want to minimise, which varies with the model parameters ($w$ & $b$ for linear regression):

![](images/gradient_descent_intuition.jpg)

## What is gradient based optimisation all about?

Our loss is just a mathematical function that depends on the parameters of our model (for example, we used the mean squared error (MSE) loss function in the previous notebooks).
We would like to move our parameters to the point where this loss is minimised.

> If we were to evaluate the value of our loss for every possible different parameterisation of our model, we would produce a **loss surface**. 

We would like to find the lowest point on this surface. 
- At this point it will have a gradient (steepness) of zero with respect to the parameters.
- As our parameters move away from that minima in some direction, the gradient will increase in that direction.

To get back to the minima, we should hence __move our weights in the opposite direction to gradient__ (simply subtract it).

![](./images/grad-based-optim.jpg)

## Numerical example

Below is an example that shows the direction to shift a parameter $W$, initialised as $w=4$, for a surface given by:

$$
L=(W-2)^2
$$ 

At this point on the surface, the gradient of the loss with respect to this parameter is positive, so we should shift it in the negative direction to push it closer the the optima.

![](images/sgd_numerical_example.jpg)

Below is a more complex potential loss surface which has more than one parameter (vertical axis represents loss value, others represent parameter values). In reality, we will often have many more features, and we won't be able to visualise the loss surface as we would need more than 3 dimensions.

<img style="height: 200px" src='./images/comp-loss-surface.png'/>

> **Note: because gradient based optimisation depends on us computing the gradient of the loss function, our loss function and model must be fully differentiable (they must be a smooth, continuous function).**

## Gradient descent

Gradient descent is an iterative, gradient based optimisation technique. 

That is, it is a technique for finding the minima (or maxima) of a function, and it does so by iteratively moving the parameters downhill, in the opposite direction to the gradient of the surface.

![](images/gradient_descent_intuition.jpg)

### The learning rate, $\alpha$

We will update our parameters by shifting them in the opposite direction to the gradient. But by what amount should we shift them in that direction?

> Learning rate $\alpha$ (abbreviated often `lr` in source code) __multiplies gradient__ to decrease (usually) or increase it's magnitude

`step_size` hence is `gradient` multiplied by our `lr`.

#### Too small `lr`

There are a few things that may happen if we get it wrong:
- If `lr` is too small our convergence __might take a long time__
- We might get stuck in local minimas or saddle points (will go over that in a second)

![title](images/low-lr.jpg)

#### Too large `lr`

- If `lr` is too large, we may jump out of minimum
- Instead of __converging__ we might __diverge__

![title](images/high-lr.jpg)

So we include the learning rate to scale up/down the size of the steps. 

> The learning rate should most likely be less than 1 though (see challenges for different example)

You should play around with the learning rate and adjust it until your model converges.

![title](images/convergence.jpg)

## Local optima

Don't stress too much about it

Yes, in the case where we are trying to minimise a function with respect to 1 or 2 parameters, gradient descent is prone to getting stuck in local optima.

But most of the models that are useful in practice depend on many more parameters (neural networks can easily have millions).

> As the number of parameters increase, it becomes exponentially unlikely that any parameterisation is at a minima, but is rather a saddle point, and so there is still an indication of how to improve.

Furthermore, __in practice we often find that we don't need to find a global optima__
Local optima can be good enough to reach our required performance.

On top of this, we can attempt to counter getting stuck in local optima by using different optimisation algorithms, such as [gradient descent with (Nesterov) momentum](https://distill.pub/2017/momentum/).

## Gradient descent

The diagrams shown above visualise how a single parameter affects the loss. 

A model with multiple parameters (such as a weight and a bias, or multiple weights) would be optimised in the same way - we would just have more of these functions. 

> We can think of each of the graphs as a cross section through a **loss surface**. 

A loss surface is shown below which visualises how the criterion of a model might vary with both parameters.

$$
L = w_1^4 + w_2^2
$$

<img style="height: 300px" src='images/x2x4.png'/>

![](images/multivariate_sgd.jpg)

If we know the function that the loss is computed from and it is differentiable, then we can calculate the derivative of the loss with respect to our model parameters:
- by hand (pretty tiring - but we're gonna do it)
- using an automatic differentiation graph (what we're gonna do later when we get to deep learning)

After that we can iteratively move each parameter in the direction of the opposite gradient. 

## Firstly, a helper function

We'll use this code shortly to visualise how training is progressing

In [ ]:
import matplotlib.pyplot as plt
def plot_loss(losses):
    """Helper function for plotting loss against epoch"""
    plt.figure() # make a figure
    plt.ylabel('Cost')
    plt.xlabel('Epoch')
    plt.plot(losses) # plot costs
    plt.show()

## 1. The data

Run the cells below to get the data and plot it

In [1]:
!pip install aicore

     |████████████████████████████████| 24.7 MB 206 kB/s 
     |████████████████████████████████| 306 kB 90.6 MB/s 


In [5]:
from sklearn import datasets, model_selection
from aicore.ml import data
import pandas as pd
import numpy as np

# Use `data.split` in order to split the data into train, validation, test
(X_train, y_train), (X_validation, y_validation), (X_test, y_test) = data.split(
    datasets.load_boston(return_X_y=True)
)
X_train, X_validation, X_test = data.standardize_multiple(X_train, X_validation, X_test)


/home/ivanyingx/miniconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housi

## 2. The model

Here's the same model we implemented before

In [ ]:
class LinearRegression:
    def __init__(self, optimiser, n_features): # initalize parameters 
        self.w = np.random.randn(n_features) ## randomly initialise weight
        self.b = np.random.randn() ## randomly initialise bias
        self.optimiser = optimiser
        
    def predict(self, X): # how do we calculate output from an input in our model?
        ypred = X @ self.w + self.b ## make a prediction using a linear hypothesis
        return ypred # return prediction

    def fit(self, X, y):
        all_costs = [] ## initialise empty list of costs to plot later
        for epoch in range(self.optimiser.epochs): ## for this many complete runs through the dataset    

            # MAKE PREDICTIONS AND UPDATE MODEL
            predictions = self.predict(X) ## make predictions
            new_w, new_b = self.optimiser.step(self.w, self.b, X, predictions, y) ## calculate updated params
            self._update_params(new_w, new_b) ## update model weight and bias
            
            # CALCULATE LOSS FOR VISUALISING
            cost = mse_loss(predictions, y) ## compute loss 
            all_costs.append(cost) ## add cost for this batch of examples to the list of costs (for plotting)

        plot_loss(all_costs)
        print('Final cost:', cost)
        print('Weight values:', self.w)
        print('Bias values:', self.b)

    
    def _update_params(self, new_w, new_b):
        self.w = new_w ## set this instance's weights to the new weight value passed to the function
        self.b = new_b ## do the same for the bias

## 3. The criterion

Let's remind ourselves the formula:

$$
\begin{equation}
    L_{mse} = \frac{1}{N}\sum_{i}^{N}(\hat{y_i} - y_i)^2
\end{equation}
$$

In [ ]:
def mse_loss(y_hat, labels): # define our criterion (loss function)
    errors = y_hat - labels ## calculate errors
    squared_errors = errors ** 2 ## square errors
    mean_squared_error = sum(squared_errors) / len(squared_errors) ## calculate mean 
    return mean_squared_error # return loss

## 4. The optimiser: Gradient descent

With linear regression, you can swap out different optimisers and stick with the same model, data and criterion.

### Implementing gradient descent from scratch

Below is a derivation for computing the rate of change (gradient) of the loss with respect to our model parameters when using a linear model and the mean squared error loss function.
![title](images/NN1_single_grad_calc.jpg)

Complete the class below to return the derivative of our loss w.r.t the weight and bias by implementing the above equations in code.

In [ ]:
import numpy as np

class SGDOptimiser:
    def __init__(self, lr, epochs):
        self.lr = lr
        self.epochs = epochs

    def _calc_deriv(self, features, predictions, labels):
        m = len(labels) ## m = number of examples
        diffs = predictions - labels ## calculate errors
        dLdw = 2 * np.sum(features.T * diffs).T / m ## calculate derivative of loss with respect to weights
        dLdb = 2 * np.sum(diffs) / m ## calculate derivative of loss with respect to bias
        return dLdw, dLdb ## return rate of change of loss wrt w and wrt b

    def step(self, w, b, features, predictions, labels):
        dLdw, dLdb = self._calc_deriv(features, predictions, labels)
        new_w = w - self.lr * dLdw
        new_b = b - self.lr * dLdb
        return new_w, new_b
    

## Putting it all together



In [ ]:
num_epochs = 1000
learning_rate = 0.001

optimiser = SGDOptimiser(lr=learning_rate, epochs=num_epochs)
model = LinearRegression(optimiser=optimiser, n_features=X_train.shape[1])
model.fit(X_train, y_train)

## `sklearn` example

`sklearn` packs everything we just did above into it's simple [LinearRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) API.

In [ ]:
from sklearn.linear_model import LinearRegression

linear_regression_model = LinearRegression() ## instantiate the linear regression model

In [ ]:
def calculate_loss(model, X, y):
    return mse_loss(
        model.predict(X),
        y
    )

print(f"Training loss before fit: {calculate_loss(model, X_train, y_train)}")
print(
    f"Validation loss before fit: {calculate_loss(model, X_validation, y_validation)}"
)
print(f"Test loss before fit: {calculate_loss(model, X_validation, y_validation)}")

In [ ]:
model = linear_regression_model.fit(X_train, y_train) ## fit the model

Now, we will do the same but fit the model for some epochs and see the loss after training, validation and test:

In [ ]:
epochs = 10000
model.fit(X_train, y_train)

print(f"Training loss after fit: {calculate_loss(model, X_train, y_train)}")
print(f"Validation loss after fit: {calculate_loss(model, X_validation, y_validation)}")
print(f"Test loss after fit: {calculate_loss(model, X_validation, y_validation)}")

We can also take a look into the final parameters of the model when using sklearn

In [ ]:
print('final weights:', model.coef_)
print('final bias:', model.intercept_)

## Why gradient based optimisation?

> Gradient based optimisation uses __heuristics__ which indicate how to improve w.r.t. loss (e.g. to minimize it).

There are other available options (like analytical solutions, sometimes), and we went through the shortcomings.

We could also search parameters __randomly__, but

- our search region may not contain an optimal parameterisation for our model. For example, if we allowed bias `[-10, 10]` we would never obtain the solution 
- exponential increase in runtime with each additional parameter
- no feedback from the process (gradient is our feedback here)

__There is still one question left unanswered__:

> What should we do when our data does not fit into memory?

## Why not pass the whole dataset through each prediction?

We know that to perform gradient based optimisation we need to pass inputs through the model (forward pass), and then compute the loss and find how it changes with respect to each of our model's parameters (backward pass). 

Modern datasets can be absolutely huge. This means that the forward pass can take a long time, as the function which our model represents has to be applied to each and every input given to it for a forward pass.

> Passing the full dataset through the model at each pass is called **full batch gradient descent**.

### Disadvantages

- Whole dataset may harm generalization and lead to overfitting
- Might be slower (slower memory access, more cache misses)


## Why not a single datapoint for each update?

Let's assume we will pass a single example to our network and backpropagate based on that.
Here is what will probably happen:
- `gradient` will vary __A LOT__ - single example is usually uninformative for our whole task
- `outliers` (special data points, which could as well be noise and are totally non-representative of the task) will affect our dataset way more

Passing single examples through the model at each pass is called **online stochastic gradient descent**.

What happens if, for some reason (memory constraints or examples come in as stream), we have to go with this approach?

## Mini-batch gradient descent

The modern way to do training is neither whole dataset nor fully stochastic (single datapoint). 

Instead we use mini-batch training:

> Sample several (usually `64-2048`, depending on memory) datapoints to compute a sample of the gradient

Most optimisation algorithms converge much faster if they are allowed to rapidly compute approximate gradients rather than slowly compute exact gradients. 

The size of the mini-batch is called the **batch size** and this technique is currently most widely used (especially in neural network).

### Advantages of mini-batch gradient descent

- We can adjust the size to fit memory on most machines
- Is faster (easier to parallelize if multiple of `2`)
- Improves generalization as each batch is a little noisy

## Why shuffle data?

It is especially important for __large and more complex models__ (neural networks). If we didn't do that we might risk the following:

- same updates are provided to the model at each batch. We want to estimate the total average, hence batches have to be different
- model "memorizes" batch (happens in neural networks)

## Poor conditioning

Different features in different datasets can have different ranges
- some features can be binary or between `[0, 1]`
- others has values ranging in hundreds or even thousands

This is problematic to most of machine learning models.

### Why?

When a small change in weight connected to feature with large values changes, the output changes significantly. This gives that weight, and hence that feature, more influence just because it is larger. 

This makes the loss function look like the example below, where it is very steep in one direction, and shallow in another.

![](images/unnormalised.png)

In the steep direction, the learning rate will need to be small enough to prevent a diverging optimisation, but because gradient signal in the other direction is so small, it wont make any progress in that direction.

> If features are on different orders of magnitude, the maximum learning rate that works will be too small to make progress in every direction.

### Example

Let's take two weights `a` and `b` and single example `x` with two features:
- first has value `0.1`
- second has value `1000`

Now, formula for linear regression would be the following:

$$
    \hat{y} = 0.1a + 1000b
$$

Now, let's see impact of `a` and `b` on $\hat{y}$:
- $a = 10, b = 0.001$ - `a` and `b` have the same impact on $\hat{y}$
- $a = 1, b = 1$ - `b` has `10000` times (!!!) more impact on $\hat{y}$

It is unlikely `a` has `10000` times less impact on the value we want to predict (and is unlikely in real world).

> We should assume all variables are __equally important unless we verify them__ via statistical testing or other measures

The range of values __is not an important factor__, relative differences between values are.

### Possible solution

One idea would be that we can use a different learning rate for each weight. But this will mean we have to search for the correct learning rate as many times as we have parameters! In many cases, examples can have a large number of features (in images, each pixel is a feature!).

### A better idea: normalisation or standardisation

We can normalize our data, which means:

> Normalization is a process of bringing features to the same value range

This ensures that relative difference between values for each feature are important, not the scale.

> We should always normalize our features (unless they are not continuous)

## Normalization & standardization

There are a lot of schemes to put values in the `[0, 1]` range. Here we will use `minmax` approach
We can do this by subtracting the minimum then dividing by the range (feature normalisation).

![title](images/normalisation.jpg)

We can alternatively use a similar method called standardisation, where we subtract the mean then divide by the standard deviation.

![](images/standardisation.jpg)

Feature normalisation puts gradients of each different model parameter on the same order of magnitude. This converts loss surfaces that might look like *valleys* into loss surfaces that look like *bowls*. Feature normalisation means that we should be able to make progress with optimisation for all model parameters using the same learning rate.

![](images/bowl.png)

> Always normalize and sanitize your input data

## Normalisation Gotchas

### 1. Data Leakage

Statistics computed from the unsplit dataset will contain information about every example.

Normalising dataset splits based on such statistics will leak information about the test and validation sets.

> Always split your data before normalising it. 

### 2. Training testing skew

This is when your training data doesn't look like your testing data. One way to cause this skew is by normalising your validation or testing sets using their own mean and standard deviation, rather than the same ones which your training data was normalised with.  If the other sets have different statistics, your model may recieve inputs that look rather different to what it was trained to make predictions from.

> All sets of data should be normalised using the same statistics.

Normlise your validation and test sets using the mean and standard deviation from your training set.

## Example

Here's a function we can use to normalise our data. Notice how it will compute the statistics if they are not yet given (as it should for the training set), but will otherwise allow you to pass them int (as you should do for the other sets)

In [ ]:
def standardize_data(dataset, mean=None, std=None):
    if mean is None and std is None:
        mean, std = np.mean(dataset, axis=0), np.std(
            dataset, axis=0
        )  ## get mean and standard deviation of dataset
    standardized_dataset = (dataset - mean) / std
    return standardized_dataset, (mean, std)

X_train, (mean, std) = standardize_data(X_train)

## Summary
- __Storchastic Gradient Descent (SGD)__ is a basic optimization technique which simply subtracts gradient from parameter's value __multiplied by `learning rate`__
- __Learning rate (`lr`)__ decides the magnitude of step taken by optimizer and is present in most optimizers
- __Mini-batch stochastic gradient descent__ is when you take parts of dataset and push through your model instead of single example or whole dataset at once. Saves memory and helps in generalization
- __Shuffling is necessary for mini-batches__ as it better mimicks overall gradient of the dataset and makes model resistant to noise
- Normalization or standardisation almost always improves our scores __sometimes our solution will diverge without it!__